## A demo using Hydrogen Hamiltonian with GPT-QE.

In [1]:
import torch
from gqe.mingpt.utils import set_seed

set_seed(3407)

In [2]:
from qwrapper.operator import PauliObservable
from gqe.mingpt.cost import EnergyCost
from qswift.compiler import DefaultOperatorPool
from benchmark.molecule import DiatomicMolecularHamiltonian
from gqe.operator_pool.uccsd import UCCSD, do_generate_molecule
from gqe.common.initializer import HFStateInitializer
from gqe.util import get_device
from gqe.mingpt.callback import DefaultCallback, PrintMonitor, FileMonitor

# molecule = generate_molecule("Li", "H", 1.596, "sto-3g", bravyi_kitaev=False)
bond_length = 3.0
geometry = f"H 0.0 0.0 0.0\n" + f"Be 0.0 0.0 {bond_length}\n" + f"H 0.0 0.0 {2 * bond_length}\n"
molecule = do_generate_molecule(geometry, "sto-3g", bravyi_kitaev=False)
nqubit = 12

# prepare Hamiltonian
hamiltonian = DiatomicMolecularHamiltonian(nqubit, molecule, bravyi_kitaev=False)

# prepare operator_pool
uccsd = UCCSD(nqubit, molecule)
paulis = uccsd.paulis
paulis.append(PauliObservable("IIIIIIIIIIII"))
print('paulis', paulis)
num_operators = len(paulis)
initializer = HFStateInitializer(n_electrons=4)
pool = DefaultOperatorPool(paulis)
cost = EnergyCost(hamiltonian, initializer, pool,
                  [1 / 320, -1 / 320, 1 / 160, -1 / 160, 1 / 80, -1 / 80, 1 / 40, -1 / 40, 0.05, -0.05, 0.1, -0.1, 0.2,
                   -0.2])


converged SCF energy = -15.0242100060364
paulis [+IIIXIIIIIIII, +IIIXZYIIIIII, +IIIXZZZYIIII, +IIIXZZZZZYII, +IIIXZZZZZZZY, +IIIYZXIIIIII, +IIIYZZZXIIII, +IIIYZZZZZXII, +IIIYZZZZZZZX, +IIXIIIIIIIII, +IIXXIIIIIIXY, +IIXXIIIIIIYX, +IIXXIIIIXYII, +IIXXIIIIYXII, +IIXXIIXYIIII, +IIXXIIYXIIII, +IIXXXYIIIIII, +IIXXYXIIIIII, +IIXYIIIIIIXX, +IIXYIIIIIIYY, +IIXYIIIIXXII, +IIXYIIIIYYII, +IIXYIIXXIIII, +IIXYIIYYIIII, +IIXYXXIIIIII, +IIXYYYIIIIII, +IIXZYIIIIIII, +IIXZZZYIIIII, +IIXZZZZZYIII, +IIXZZZZZZZYI, +IIYXIIIIIIXX, +IIYXIIIIIIYY, +IIYXIIIIXXII, +IIYXIIIIYYII, +IIYXIIXXIIII, +IIYXIIYYIIII, +IIYXXXIIIIII, +IIYXYYIIIIII, +IIYYIIIIIIXY, +IIYYIIIIIIYX, +IIYYIIIIXYII, +IIYYIIIIYXII, +IIYYIIXYIIII, +IIYYIIYXIIII, +IIYYXYIIIIII, +IIYYYXIIIIII, +IIYZXIIIIIII, +IIYZZZXIIIII, +IIYZZZZZXIII, +IIYZZZZZZZXI, +IXIIIIIIIIII, +IXXIIXZZZZYI, +IXXIIYZZZZXI, +IXXIXZZZZZZY, +IXXIYZZZZZZX, +IXYIIXZZZZXI, +IXYIIYZZZZYI, +IXYIXZZZZZZX, +IXYIYZZZZZZY, +IXZXIXZZZZZY, +IXZXIYZZZZZX, +IXZYIXZZZZZX, +IXZYIYZZZZZY, +IXZZZ

## FCI energy by diagonalization

In [3]:
from qwrapper.hamiltonian import compute_ground_state

#print(compute_ground_state(hamiltonian))

In [4]:
print("hf state:", hamiltonian.exact_value(initializer.init_circuit(12, [], "qulacs")))

hf state: -15.024210006036476


## Setup for GPT

In [5]:
# create a GPT instance
from gqe.mingpt.model import GPT

model_config = GPT.get_default_config()
model_config.model_type = 'gpt2'
model_config.vocab_size = cost.vocab_size()
model_config.n_gates = 20  # The number of gates for each circuit
model_config.block_size = 20
model_config.temperature = 5  # Each gate is generated with probability exp(-temperature * logit)
model_config.embd_pdrop = 0.1
model_config.resid_pdrop = 0.1
model_config.attn_pdrop = 0.1
model_config.std = 0.02
model_config.energy_offset = 14
model = GPT(model_config, cost)

number of parameters: 86.67M


In [6]:
# create a Trainer object
from gqe.mingpt.trainer import Trainer

train_config = Trainer.get_default_config()
train_config.learning_rate = 5e-7  # the model we're using is so small that we can go a bit faster
train_config.max_iters = 1000
train_config.num_workers = 10
train_config.n_samples = 50
trainer = Trainer(train_config, model)

running on device mps


In [7]:
file_monitor = FileMonitor()
callback_generator = DefaultCallback(model, monitors=[PrintMonitor(), file_monitor], del_temperature=0.01)
trainer.set_callback('on_batch_end', callback_generator.generate())
trainer.run()
torch.save(model.state_dict(), '../saved_models/gptqe_test_2')

iter_dt 0.00s; iter 0: train loss 0.10874 temperature: 5
mean_logits tensor([-15.0704, -14.5788, -14.7971, -14.9020, -14.9126, -15.0692, -14.9847,
        -15.1205, -15.1200, -14.9431, -15.0508, -15.0025, -15.0242, -14.9548,
        -15.1773, -14.7875, -15.0431, -15.0530, -14.9089, -14.7773, -15.1553,
        -14.8552, -14.8318, -15.1238, -14.8675, -14.8745, -14.9520, -15.0546,
        -14.7615, -14.9114, -14.8449, -14.7887, -14.7721, -14.9647, -14.8493,
        -15.0891, -15.0941, -15.0619, -14.8584, -14.9705, -14.8595, -15.0213,
        -14.9194, -14.8658, -14.8469, -15.0108, -14.9461, -14.9462, -14.6889,
        -14.9127], device='mps:0', grad_fn=<SubBackward0>)
energies: tensor([-14.8700, -15.0156, -14.9097, -14.8310, -14.8866, -14.9847, -15.0049,
        -15.0415, -15.0526, -15.0254, -14.9891, -14.9545, -15.0208, -14.9140,
        -14.9704, -14.9098, -14.8588, -14.9571, -14.9327, -14.8557, -14.9458,
        -14.9851, -14.9119, -14.9134, -14.8633, -14.8871, -14.9378, -15.0416,
    

Traceback (most recent call last):
  File "/Users/koheinakaji/PycharmProjects/gqe/venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3442, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/mw/1twnd__d5mnb201cs3fp1jn40000gn/T/ipykernel_2597/769965459.py", line 4, in <module>
    trainer.run()
  File "/Users/koheinakaji/PycharmProjects/gqe/gqe/mingpt/trainer.py", line 71, in run
    self.loss, detail = model(input)
  File "/Users/koheinakaji/PycharmProjects/gqe/venv/lib/python3.8/site-packages/torch/nn/modules/module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
  File "/Users/koheinakaji/PycharmProjects/gqe/gqe/mingpt/model.py", line 341, in forward
  File "/Users/koheinakaji/PycharmProjects/gqe/gqe/mingpt/model.py", line 294, in cost
    idx_output, logits_tensor = self.generate(idx, self.n_gates)
  File "/Users/koheinakaji/PycharmProjects/gqe/gqe/mingpt/model.py", line 317, in generate
    logits_base

In [ ]:
print(file_monitor.min_energy)


In [ ]:
cost.sequence.tool = "qiskit"
print(file_monitor.min_indices)
cost.sequence._get_circuit(file_monitor.min_indices).qc.draw(output="mpl", plot_barriers=True)